In [10]:
!pip install rasterio
!pip install xdem


In [11]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import rasterio
from rasterio.plot import show
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import os
import xdem

In [12]:
import math

def margin_from_zoom(lat, zoom, image_size_pixels):
    """
    Calculate margin in degrees to match Google Maps zoom level for given latitude and image size.

    This uses the Web Mercator projection formula that Google Maps uses.
    Returns a margin in degrees for both latitude and longitude.
    """

    # Google Maps uses Web Mercator projection (EPSG:3857)
    # At zoom level z, the map shows a region of width 360/2^z degrees longitude
    # and a corresponding height in latitude (which varies with latitude due to Mercator projection)

    # For a 256x256 tile at zoom level z, the tile covers:
    # - Longitude: 360 / (2^z) degrees
    # - The latitude coverage depends on the Mercator projection

    # Calculate the ground resolution (meters per pixel) at the given latitude and zoom
    # This is the standard Web Mercator formula
    cos_lat = math.cos(math.radians(lat))
    ground_resolution = (156543.03392 * cos_lat) / (2 ** zoom)

    # Convert the image size from pixels to meters
    image_width_meters = image_size_pixels * ground_resolution
    image_height_meters = image_size_pixels * ground_resolution

    # Convert meters to degrees
    # Latitude: 1 degree ≈ 111,320 meters (constant)
    meters_per_degree_lat = 111320.0
    margin_lat = image_height_meters / (2 * meters_per_degree_lat)

    # Longitude: varies with latitude
    meters_per_degree_lon = meters_per_degree_lat * cos_lat
    margin_lon = image_width_meters / (2 * meters_per_degree_lon)

    return margin_lat, margin_lon

In [13]:
def overlay_maps_and_dem(google_map_image, dem_path, lat_center, lon_center, zoom_level,
                        image_size_pixels=600, alpha_dem=0.6, alpha_map=0.8):
    """
    Overlay Google Static Maps image with DEM data for alignment verification.

    Parameters:
    - google_map_image: PIL Image or numpy array of the Google Maps image
    - dem_path: path to the DEM TIFF file
    - lat_center, lon_center: center coordinates
    - zoom_level: zoom level used
    - image_size_pixels: size of the images
    - alpha_dem: transparency of DEM overlay (0-1)
    - alpha_map: transparency of map image (0-1)
    """

    # Calculate the bounds used for both images
    margin_lat, margin_lon = margin_from_zoom(lat_center, zoom_level, image_size_pixels)

    bounds = {
        'south': lat_center - margin_lat,
        'north': lat_center + margin_lat,
        'west': lon_center - margin_lon,
        'east': lon_center + margin_lon
    }

    print(f"Image bounds:")
    print(f"  Latitude: {bounds['south']:.6f} to {bounds['north']:.6f}")
    print(f"  Longitude: {bounds['west']:.6f} to {bounds['east']:.6f}")

    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Map and DEM Overlay - Zoom {zoom_level}\nCenter: {lat_center:.6f}, {lon_center:.6f}',
                fontsize=14)

    # Convert Google Maps image to numpy array if it's PIL
    if hasattr(google_map_image, 'convert'):
        map_array = np.array(google_map_image.convert('RGB'))
    else:
        map_array = google_map_image

    # 1. Show Google Maps image
    axes[0,0].imshow(map_array)
    axes[0,0].set_title('Google Static Maps')
    axes[0,0].set_xlabel('Pixels')
    axes[0,0].set_ylabel('Pixels')
    axes[0,0].grid(True, alpha=0.3)

    # Add center crosshair
    center_x, center_y = image_size_pixels // 2, image_size_pixels // 2
    axes[0,0].axhline(y=center_y, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[0,0].axvline(x=center_x, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[0,0].plot(center_x, center_y, 'ro', markersize=8, label='Center')
    axes[0,0].legend()

    # 2. Show DEM data
    with rasterio.open(dem_path) as dem_dataset:
        dem_data = dem_dataset.read(1)
        dem_bounds = dem_dataset.bounds
        dem_crs = dem_dataset.crs

        print(f"DEM bounds: {dem_bounds}")
        print(f"DEM CRS: {dem_crs}")
        print(f"DEM shape: {dem_data.shape}")

        # Create elevation colormap
        dem_masked = np.ma.masked_where(dem_data == -9999, dem_data)

        im1 = axes[0,1].imshow(dem_masked, cmap='terrain', aspect='equal')
        axes[0,1].set_title(f'DEM Elevation Data\nMin: {np.nanmin(dem_data):.1f}m, Max: {np.nanmax(dem_data):.1f}m')
        axes[0,1].set_xlabel('Pixels')
        axes[0,1].set_ylabel('Pixels')
        plt.colorbar(im1, ax=axes[0,1], label='Elevation (m)')

        # Add center crosshair to DEM
        dem_center_x, dem_center_y = dem_data.shape[1] // 2, dem_data.shape[0] // 2
        axes[0,1].axhline(y=dem_center_y, color='red', linestyle='--', alpha=0.7, linewidth=2)
        axes[0,1].axvline(x=dem_center_x, color='red', linestyle='--', alpha=0.7, linewidth=2)
        axes[0,1].plot(dem_center_x, dem_center_y, 'ro', markersize=8, label='Center')
        axes[0,1].legend()

    # 3. Overlay both images (resized to match)
    # Resize DEM to match Google Maps image size
    from PIL import Image as PILImage
    dem_pil = PILImage.fromarray(dem_masked.filled(0).astype(np.uint8))
    dem_resized = dem_pil.resize((image_size_pixels, image_size_pixels), PILImage.LANCZOS)
    dem_resized_array = np.array(dem_resized)

    # Create overlay
    axes[1,0].imshow(map_array, alpha=alpha_map)

    # Create a custom colormap for the DEM overlay
    colors = ['blue', 'green', 'yellow', 'red']
    n_bins = 256
    cmap = LinearSegmentedColormap.from_list('elevation', colors, N=n_bins)

    # Normalize DEM data for overlay
    dem_normalized = (dem_resized_array - dem_resized_array.min()) / (dem_resized_array.max() - dem_resized_array.min())

    im2 = axes[1,0].imshow(dem_normalized, cmap=cmap, alpha=alpha_dem)
    axes[1,0].set_title('Overlay: Map + DEM')
    axes[1,0].set_xlabel('Pixels')
    axes[1,0].set_ylabel('Pixels')

    # Add center crosshair
    axes[1,0].axhline(y=center_y, color='red', linestyle='--', alpha=0.9, linewidth=2)
    axes[1,0].axvline(x=center_x, color='red', linestyle='--', alpha=0.9, linewidth=2)
    axes[1,0].plot(center_x, center_y, 'ro', markersize=8, label='Center')
    axes[1,0].legend()

    # 4. Side-by-side comparison with transparency
    axes[1,1].imshow(map_array, alpha=0.7)
    axes[1,1].imshow(dem_normalized, cmap='terrain', alpha=0.5)
    axes[1,1].set_title('Blended Overlay')
    axes[1,1].set_xlabel('Pixels')
    axes[1,1].set_ylabel('Pixels')

    # Add grid for easier alignment checking
    for ax in axes.flat:
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')

    plt.tight_layout()
    plt.show()

    # Print alignment information
    print(f"\nAlignment Check:")
    print(f"Google Maps image size: {map_array.shape}")
    print(f"DEM original size: {dem_data.shape}")
    print(f"Image center pixel: ({center_x}, {center_y})")
    print(f"Calculated margins: lat={margin_lat:.6f}°, lon={margin_lon:.6f}°")

    return fig


def quick_overlay(google_map_image, dem_path, alpha_dem=0.6):
    """
    Quick overlay function for fast checking
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Convert Google Maps image
    if hasattr(google_map_image, 'convert'):
        map_array = np.array(google_map_image.convert('RGB'))
    else:
        map_array = google_map_image

    # Load and resize DEM
    with rasterio.open(dem_path) as dem_dataset:
        dem_data = dem_dataset.read(1)
        dem_masked = np.ma.masked_where(dem_data == -9999, dem_data)

    # Resize DEM to match map
    from PIL import Image as PILImage
    dem_pil = PILImage.fromarray(((dem_masked - dem_masked.min()) / (dem_masked.max() - dem_masked.min()) * 255).filled(0).astype(np.uint8))
    dem_resized = dem_pil.resize(map_array.shape[:2][::-1], PILImage.LANCZOS)
    dem_array = np.array(dem_resized)

    # Overlay
    ax.imshow(map_array, alpha=0.8)
    ax.imshow(dem_array, cmap='terrain', alpha=alpha_dem)

    # Add center crosshairs
    center_x, center_y = map_array.shape[1] // 2, map_array.shape[0] // 2
    ax.axhline(y=center_y, color='red', linestyle='--', alpha=0.9, linewidth=2)
    ax.axvline(x=center_x, color='red', linestyle='--', alpha=0.9, linewidth=2)
    ax.plot(center_x, center_y, 'ro', markersize=10)

    ax.set_title('Quick Overlay Check')
    ax.grid(True, alpha=0.3)
    plt.show()

    return fig


In [16]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import xdem
import os

def overlay_map_and_slope(google_map_image, dem_path, lat_center, lon_center, zoom_level,
                         image_size_pixels=600, alpha_slope=0.6, alpha_map=0.8,
                         terrain_attribute="slope"):
    """
    Overlay Google Static Maps image with DEM terrain attributes (slope, aspect, hillshade, etc.)

    Parameters:
    - google_map_image: PIL Image or numpy array of the Google Maps image
    - dem_path: path to the DEM TIFF file
    - lat_center, lon_center: center coordinates
    - zoom_level: zoom level used
    - image_size_pixels: size of the images
    - alpha_slope: transparency of terrain overlay (0-1)
    - alpha_map: transparency of map image (0-1)
    - terrain_attribute: "slope", "aspect", "hillshade", "curvature", or "all"
    """

    # Calculate the bounds used for both images
    margin_lat, margin_lon = margin_from_zoom(lat_center, zoom_level, image_size_pixels)

    bounds = {
        'south': lat_center - margin_lat,
        'north': lat_center + margin_lat,
        'west': lon_center - margin_lon,
        'east': lon_center + margin_lon
    }

    print(f"Image bounds:")
    print(f"  Latitude: {bounds['south']:.6f} to {bounds['north']:.6f}")
    print(f"  Longitude: {bounds['west']:.6f} to {bounds['east']:.6f}")

    # Load the DEM and compute terrain attributes
    print("Loading DEM and computing terrain attributes...")
    dem = xdem.DEM(dem_path)

    # Compute terrain attributes
    terrain_data = {}
    terrain_cmaps = {}
    terrain_titles = {}

    if terrain_attribute == "slope" or terrain_attribute == "all":
        slope = xdem.terrain.slope(dem, method="ZevenbergThorne")
        terrain_data['slope'] = slope.data
        terrain_cmaps['slope'] = 'Reds'
        terrain_titles['slope'] = 'Slope (°)'

    if terrain_attribute == "aspect" or terrain_attribute == "all":
        aspect = dem.aspect()
        terrain_data['aspect'] = aspect.data
        terrain_cmaps['aspect'] = 'twilight'
        terrain_titles['aspect'] = 'Aspect (°)'

    if terrain_attribute == "hillshade" or terrain_attribute == "all":
        hillshade = dem.hillshade()
        terrain_data['hillshade'] = hillshade.data
        terrain_cmaps['hillshade'] = 'gray'
        terrain_titles['hillshade'] = 'Hillshade'

    if terrain_attribute == "curvature" or terrain_attribute == "all":
        curvature = dem.curvature()
        terrain_data['curvature'] = curvature.data
        terrain_cmaps['curvature'] = 'coolwarm'
        terrain_titles['curvature'] = 'Curvature'

    # Convert Google Maps image to numpy array if it's PIL
    if hasattr(google_map_image, 'convert'):
        map_array = np.array(google_map_image.convert('RGB'))
    else:
        map_array = google_map_image

    # Create figure layout based on what we're showing
    if terrain_attribute == "all":
        fig, axes = plt.subplots(3, 2, figsize=(15, 18))
        axes = axes.flatten()
    else:
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()

    fig.suptitle(f'Google Maps + DEM {terrain_attribute.title()} Overlay - Zoom {zoom_level}\n'
                f'Center: {lat_center:.6f}, {lon_center:.6f}', fontsize=14)

    # 1. Show Google Maps image
    axes[0].imshow(map_array)
    axes[0].set_title('Google Static Maps')
    axes[0].set_xlabel('Pixels')
    axes[0].set_ylabel('Pixels')
    axes[0].grid(True, alpha=0.3)

    # Add center crosshair
    center_x, center_y = image_size_pixels // 2, image_size_pixels // 2
    axes[0].axhline(y=center_y, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[0].axvline(x=center_x, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[0].plot(center_x, center_y, 'ro', markersize=8, label='Center')
    axes[0].legend()

    # 2. Show primary terrain attribute
    primary_attr = terrain_attribute if terrain_attribute != "all" else "slope"
    terrain_array = terrain_data[primary_attr]

    # Mask invalid values
    terrain_masked = np.ma.masked_invalid(terrain_array)

    im1 = axes[1].imshow(terrain_masked, cmap=terrain_cmaps[primary_attr], aspect='equal')
    axes[1].set_title(f'{terrain_titles[primary_attr]}\n'
                     f'Min: {np.nanmin(terrain_array):.1f}, Max: {np.nanmax(terrain_array):.1f}')
    axes[1].set_xlabel('Pixels')
    axes[1].set_ylabel('Pixels')
    plt.colorbar(im1, ax=axes[1], label=terrain_titles[primary_attr])

    # Add center crosshair
    terrain_center_x = terrain_array.shape[1] // 2
    terrain_center_y = terrain_array.shape[0] // 2
    axes[1].axhline(y=terrain_center_y, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[1].axvline(x=terrain_center_x, color='red', linestyle='--', alpha=0.7, linewidth=2)
    axes[1].plot(terrain_center_x, terrain_center_y, 'ro', markersize=8, label='Center')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # 3. Create overlay - resize terrain data to match Google Maps
    from PIL import Image as PILImage

    # Normalize and resize terrain data
    terrain_normalized = (terrain_masked - np.nanmin(terrain_masked)) / (np.nanmax(terrain_masked) - np.nanmin(terrain_masked))
    terrain_normalized = terrain_normalized.filled(0)  # Fill masked values with 0

    # Convert to PIL and resize
    terrain_uint8 = (terrain_normalized * 255).astype(np.uint8)
    terrain_pil = PILImage.fromarray(terrain_uint8)
    terrain_resized = terrain_pil.resize((image_size_pixels, image_size_pixels), PILImage.LANCZOS)
    terrain_resized_array = np.array(terrain_resized) / 255.0

    # Create overlay
    axes[2].imshow(map_array, alpha=alpha_map)
    im2 = axes[2].imshow(terrain_resized_array, cmap=terrain_cmaps[primary_attr], alpha=alpha_slope)
    axes[2].set_title(f'Overlay: Map + {terrain_titles[primary_attr]}')
    axes[2].set_xlabel('Pixels')
    axes[2].set_ylabel('Pixels')

    # Add center crosshair
    axes[2].axhline(y=center_y, color='red', linestyle='--', alpha=0.9, linewidth=2)
    axes[2].axvline(x=center_x, color='red', linestyle='--', alpha=0.9, linewidth=2)
    axes[2].plot(center_x, center_y, 'ro', markersize=8, label='Center')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    # 4. Blended version
    axes[3].imshow(map_array, alpha=0.7)
    axes[3].imshow(terrain_resized_array, cmap=terrain_cmaps[primary_attr], alpha=0.5)
    axes[3].set_title(f'Blended: Map + {terrain_titles[primary_attr]}')
    axes[3].set_xlabel('Pixels')
    axes[3].set_ylabel('Pixels')
    axes[3].grid(True, alpha=0.3)

    # If showing all attributes, add more plots
    if terrain_attribute == "all" and len(axes) > 4:
        attr_list = list(terrain_data.keys())
        for i, attr in enumerate(attr_list[1:], start=4):  # Skip first one (already shown)
            if i < len(axes):
                terrain_array_extra = terrain_data[attr]
                terrain_masked_extra = np.ma.masked_invalid(terrain_array_extra)

                im_extra = axes[i].imshow(terrain_masked_extra, cmap=terrain_cmaps[attr], aspect='equal')
                axes[i].set_title(f'{terrain_titles[attr]}')
                axes[i].set_xlabel('Pixels')
                axes[i].set_ylabel('Pixels')
                plt.colorbar(im_extra, ax=axes[i], label=terrain_titles[attr])
                axes[i].grid(True, alpha=0.3)

    # Hide unused axes
    for i in range(len(terrain_data) + 2, len(axes)):
        axes[i].set_visible(False)

    plt.tight_layout()
    plt.show()

    # Print alignment information
    print(f"\nAlignment Check:")
    print(f"Google Maps image size: {map_array.shape}")
    print(f"DEM/Terrain original size: {terrain_array.shape}")
    print(f"Image center pixel: ({center_x}, {center_y})")
    print(f"Calculated margins: lat={margin_lat:.6f}°, lon={margin_lon:.6f}°")

    return fig

In [15]:
def save_clean_overlays(google_map_image, dem_path, output_dir="./",
                       alpha_terrain=0.5, dpi=300):
    """
    Create clean overlays without axes or padding, returning PIL Images.

    Returns:
    - tuple: (slope_overlay_image, height_overlay_image, slope_path, height_path)
    """

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Convert Google Maps image
    if hasattr(google_map_image, 'convert'):
        map_array = np.array(google_map_image.convert('RGB'))
    else:
        map_array = google_map_image

    print("Processing DEM data...")

    # Load DEM and compute slope
    dem = xdem.DEM(dem_path)
    slope = xdem.terrain.slope(dem, method="ZevenbergThorne")

    elevation_data = dem.data
    slope_data = slope.data

    # Prepare terrain data
    def prepare_and_overlay(terrain_array, cmap_name):
        # Mask and normalize
        terrain_masked = np.ma.masked_invalid(terrain_array)
        terrain_normalized = (terrain_masked - np.nanmin(terrain_masked)) / (np.nanmax(terrain_masked) - np.nanmin(terrain_masked))
        terrain_normalized = terrain_normalized.filled(0)

        # Resize to match map
        terrain_uint8 = (terrain_normalized * 255).astype(np.uint8)
        terrain_pil = Image.fromarray(terrain_uint8)
        terrain_resized = terrain_pil.resize((map_array.shape[1], map_array.shape[0]), Image.LANCZOS)
        terrain_array_resized = np.array(terrain_resized) / 255.0

        # Create overlay using matplotlib to get proper colormap
        fig, ax = plt.subplots(figsize=(map_array.shape[1]/100, map_array.shape[0]/100), dpi=100)
        ax.imshow(map_array)
        ax.imshow(terrain_array_resized, cmap=cmap_name, alpha=alpha_terrain)
        ax.axis('off')
        ax.set_xlim(0, map_array.shape[1])
        ax.set_ylim(map_array.shape[0], 0)

        # Convert to numpy array
        fig.canvas.draw()
        overlay_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        overlay_array = overlay_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close(fig)

        return Image.fromarray(overlay_array)

    # Create overlays
    print("Creating slope overlay...")
    slope_overlay_img = prepare_and_overlay(slope_data, 'Reds')

    print("Creating height overlay...")
    height_overlay_img = prepare_and_overlay(elevation_data, 'terrain')

    # Save images
    slope_path = os.path.join(output_dir, "map_slope_overlay.png")
    height_path = os.path.join(output_dir, "map_height_overlay.png")

    slope_overlay_img.save(slope_path, dpi=(dpi, dpi))
    height_overlay_img.save(height_path, dpi=(dpi, dpi))

    print(f"Clean overlays saved:")
    print(f"  Slope overlay: {slope_path}")
    print(f"  Height overlay: {height_path}")

    return slope_overlay_img, height_overlay_img, slope_path, height_path
